In [173]:
import pandas as pd
import numpy as np

datapath="../data/"

submission_data = pd.read_csv(datapath + "submission.csv")
test_data = pd.read_csv(datapath + "test.csv")
train_data = pd.read_csv(datapath + "train.csv")
countrycodes=pd.read_csv(datapath+"wikipedia-iso-country-codes.csv")
indicator_data=pd.read_excel(datapath+"Indicator_list.xlsx").fillna("")
age = pd.read_excel(datapath+"API_SP.POP.65UP.TO.ZS_DS2_en_excel_v2_887753.xls")
population = pd.read_excel(datapath+"total_population.xls")
population_dens = pd.read_excel(datapath+"population_density.xls")

indicator_data["long_form"]=indicator_data["long_form"].apply(lambda x: "_".join(x.split(" ")))
train_data[["ConfirmedCases","Fatalities"]]=train_data[["ConfirmedCases","Fatalities"]].astype(int)
submission_data[["ConfirmedCases","Fatalities"]]=submission_data[["ConfirmedCases","Fatalities"]].astype(int)

In [174]:
countrycodes=countrycodes[["Alpha-3 code", "English short name lower case"]]
countrycodes=countrycodes.drop(countrycodes[countrycodes["English short name lower case"].isna()].index,axis=0).reset_index(drop=True)
countrycodes.columns = ["Shortcut", "Country_Region"]

In [176]:
data_sets = [(age,"Population ages 65 and above"), (population,"Pop"), (population_dens,"Pop_dens")]
world= []
for worldbank, name in data_sets:
    worldbank.columns = worldbank.iloc[2]
    worldbank.drop(index=[0, 1,2],inplace = True)
    worldbank["Year"] = 2018
    worldbank = worldbank[["Indicator Code", "Country Code", "Year", 2018.0]]
    worldbank.columns = ["Indicator Code", "Shortcut", "Year", name]
    worldbank.head()
    world.append(worldbank)

In [177]:
world

[        Indicator Code Shortcut  Year  Population ages 65 and above
 3    SP.POP.65UP.TO.ZS      ABW  2018                     13.550947
 4    SP.POP.65UP.TO.ZS      AFG  2018                      2.584927
 5    SP.POP.65UP.TO.ZS      AGO  2018                      2.216374
 6    SP.POP.65UP.TO.ZS      ALB  2018                     13.744736
 7    SP.POP.65UP.TO.ZS      AND  2018                           NaN
 ..                 ...      ...   ...                           ...
 262  SP.POP.65UP.TO.ZS      XKX  2018                           NaN
 263  SP.POP.65UP.TO.ZS      YEM  2018                      2.876270
 264  SP.POP.65UP.TO.ZS      ZAF  2018                      5.318005
 265  SP.POP.65UP.TO.ZS      ZMB  2018                      2.099678
 266  SP.POP.65UP.TO.ZS      ZWE  2018                      2.939524
 
 [264 rows x 4 columns],     Indicator Code Shortcut  Year         Pop
 3      SP.POP.TOTL      ABW  2018    105845.0
 4      SP.POP.TOTL      AFG  2018  37172386.0
 5   

In [153]:
import requests
from tqdm import tqdm

def import_data(indicator="",indicator_type="",name=""):
    url="https://ghoapi.azureedge.net/api/{}".format(indicator)
    data = pd.DataFrame(requests.get(url).json()["value"])
    data.drop([col for col in data.columns if data[col].isna().sum() == data.shape[0]],axis=1,inplace=True)
    data=data.drop(data[data.NumericValue.isna()].index,axis=0).reset_index(drop=True)
    data=data[data.Dim1==indicator_type] if len(indicator_type)>0 else data
    data=data[["IndicatorCode","SpatialDim","TimeDim","NumericValue"]].drop_duplicates(subset=["IndicatorCode","SpatialDim"],keep="last").reset_index(drop=True)
    data.columns=["IndicatorCode","Shortcut","Year",name]
    
    return data

def import_who_indicators(indicator_data):
    data=[import_data(indicator["short_form"],indicator["addition"],indicator["long_form"]) for _,indicator in tqdm(indicator_data.iterrows())]
    return data
    
who_data_list=import_who_indicators(indicator_data)
who_data_list.append(worldbank)

11it [00:11,  1.05s/it]


array([13.55094714,  2.58492694,  2.21637365, 13.74473591,         nan,
        4.55787642,  1.08500098, 11.11778888, 11.25381766,  8.79982552,
       15.65647523, 19.00156646,  6.19518275,  2.24694046, 18.78874374,
        3.2536053 ,  2.40698083,  5.15839064, 21.02191443,  2.42633388,
        7.25760236, 16.47031747, 14.84514817,  4.73645878,  7.19194739,
        8.92283783, 15.80269396,  4.87314757,  6.00301171,  4.22387435,
        2.82577371, 17.23200668, 18.4601185 , 18.62321656, 17.29746216,
       11.52980166, 10.92088354,  2.85894302,  2.72887736,  3.01784062,
        2.68171954,  8.47804702,  3.00700931,  4.60932716,  9.54984767,
        8.79665875, 15.18621142, 16.68489481, 13.71906177, 19.42087689,
       21.461962  ,  4.52757938, 19.81295264,  7.08281748,  6.36249654,
        9.49623787,  5.81780568, 10.744358  , 12.0469774 , 16.3488906 ,
        7.15728973,  5.22977935, 20.60028759, 19.37850752, 19.62635685,
        3.501133  , 19.93452168,  3.45084801, 21.72078755,  5.44

In [163]:
def match_country_code(countrycodes,traindata):
    result = traindata.merge(countrycodes,on="Country_Region",how="left").reset_index(drop=True)
    return result

new_traindata = match_country_code(countrycodes,train_data)

def match_dataframes(new_traindata,who_data):
    result = new_traindata.merge(who_data,on="Shortcut",how="left")
    return result

for who_data in who_data_list:
    new_traindata = match_dataframes(new_traindata,who_data)

In [164]:
new_traindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19110 entries, 0 to 19109
Data columns (total 43 columns):
Id                                                                                         19110 non-null int64
Province_State                                                                             8125 non-null object
Country_Region                                                                             19110 non-null object
Date                                                                                       19110 non-null object
ConfirmedCases                                                                             19110 non-null int32
Fatalities                                                                                 19110 non-null int32
Shortcut                                                                                   18980 non-null object
IndicatorCode_x                                                                            18850 non-nul

In [31]:
new_traindata[new_traindata.isna().any(axis=1)]

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Shortcut,IndicatorCode_x,Year_x,Population_using_at_least_basic_sanitation_services_(%),...,Raised_fasting_blood_glucose_(>=7.0_mmol/L_or_on_medication)(age-standardized_estimate),IndicatorCode_x,Year_x,Raised_blood_pressure_(SBP>=140_OR_DBP>=90)_(age-standardized_estimate),IndicatorCode_y,Year_y,"Prevalence_of_obesity_among_adults,_BMI_>=_30_(age-standardized_estimate)_(%)",IndicatorCode,Year,Health_Service_Provision
0,1,NaN,Afghanistan,2020-01-22,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
1,2,NaN,Afghanistan,2020-01-23,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
2,3,NaN,Afghanistan,2020-01-24,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
3,4,NaN,Afghanistan,2020-01-25,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
4,5,NaN,Afghanistan,2020-01-26,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19105,29361,NaN,Zimbabwe,2020-03-22,3,0,ZWE,WSH_SANITATION_BASIC,2017.0,46.21599,...,7.1,BP_04,2015.0,28.2,NCD_BMI_30A,2016.0,4.7,IHRSPAR_CAPACITY09,2018.0,20.0
19106,29362,NaN,Zimbabwe,2020-03-23,3,1,ZWE,WSH_SANITATION_BASIC,2017.0,46.21599,...,7.1,BP_04,2015.0,28.2,NCD_BMI_30A,2016.0,4.7,IHRSPAR_CAPACITY09,2018.0,20.0
19107,29363,NaN,Zimbabwe,2020-03-24,3,1,ZWE,WSH_SANITATION_BASIC,2017.0,46.21599,...,7.1,BP_04,2015.0,28.2,NCD_BMI_30A,2016.0,4.7,IHRSPAR_CAPACITY09,2018.0,20.0
19108,29364,NaN,Zimbabwe,2020-03-25,3,1,ZWE,WSH_SANITATION_BASIC,2017.0,46.21599,...,7.1,BP_04,2015.0,28.2,NCD_BMI_30A,2016.0,4.7,IHRSPAR_CAPACITY09,2018.0,20.0


In [127]:
new_traindata[new_traindata["Shortcut"].isna()]["Country_Region"].unique()

array(['Diamond Princess', 'Holy See'], dtype=object)

In [40]:
countrycodes[countrycodes["ISO3166-1-Alpha-3"]=="BOL"]

,official_name_ar,official_name_cn,official_name_en,official_name_es,official_name_fr,official_name_ru,ISO3166-1-Alpha-2,ISO3166-1-Alpha-3,ISO3166-1-numeric,ISO4217-currency_alphabetic_code,...,Least Developed Countries (LDC),MARC,Region Code,Region Name,Small Island Developing States (SIDS),Sub-region Code,Sub-region Name,TLD,WMO,is_independent
26,بوليفيا (دولة - المتعددة القوميات),玻利维亚(多民族国),Bolivia (Plurinational State of),Bolivia (Estado Plurinacional de),Bolivie (État plurinational de),Боливия (Многонациональное Государство),BO,BOL,68.0,BOB,...,NaN,bo,19.0,Americas,NaN,419.0,Latin America and the Caribbean,.bo,BO,Yes


In [21]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19110 entries, 0 to 19109
Data columns (total 6 columns):
Id                19110 non-null int64
Province_State    8125 non-null object
Country_Region    19110 non-null object
Date              19110 non-null object
ConfirmedCases    19110 non-null int32
Fatalities        19110 non-null int32
dtypes: int32(2), int64(1), object(3)
memory usage: 746.6+ KB


In [8]:
[data.Year for data in who_data_list]

[0      2017
 1      2017
 2      2017
 3      2017
 4      2017
        ... 
 189    2017
 190    2017
 191    2017
 192    2017
 193    2017
 Name: Year, Length: 194, dtype: int64, 0      2011
 1      2015
 2      2012
 3      2010
 4      2006
        ... 
 175    2007
 176    2014
 177    2010
 178    2008
 179    2006
 Name: Year, Length: 180, dtype: int64, 0      2014
 1      2014
 2      2014
 3      2014
 4      2014
        ... 
 198    2014
 199    2014
 200    2014
 201    2014
 202    2014
 Name: Year, Length: 203, dtype: int64, 0      2001
 1      1997
 2      2010
 3      2003
 4      2000
        ... 
 188    1999
 189    1997
 190    2004
 191    2004
 192    1990
 Name: Year, Length: 193, dtype: int64, 0      2000
 1      2005
 2      2005
 3      2001
 4      2008
        ... 
 187    2006
 188    1999
 189    2017
 190    2015
 191    2017
 Name: Year, Length: 192, dtype: int64, 0      2007
 1      2009
 2      2016
 3      2012
 4      2012
        ... 
 180    2014

In [249]:
list(set(who_data_list[6].Year))

[2025, 2012]